In [ ]:
# Signature: wa.process_raw_data(raw_ind_ret, extreme_num=3, extreme_method='mad', scale_method='normal', funcs=None)
# Source:   
def process_raw_data(raw_ind_ret, extreme_num=3, extreme_method='mad', scale_method='normal', funcs=None):
    """
    处理原始因子数据

    :param raw_ind_ret:    pd.DataFrame, 原始因子数据，结构如prepare_raw_data返回的数据
    :param extreme_num:             int, 去极值的判断区间，如果extreme_method='std',extreme_num=3,则超过3个标准差的为极端值
    :param extreme_method:          str, 去极值的方法，可选参数'mad'：平均绝对离差法,'std':标准差法, 默认'mad'
    :param scale_method:            str, 标准化的方法，可选参数'normal': 正常标准化，因子均值为因子算法平均值；
                                                               'cap': 市值加权标准化,因子均值为市值加权均值。默认'normal'

    :param funcs:          list, 自定义数据处理函数，默认None
    :return:               pd.DataFrame
    """
    from functools import partial
    raw_data = raw_ind_ret.copy()

    if extreme_method and extreme_method not in ['mad','std']:
        raise ValueError("extreme_method must be one of ['mad','std'] or False")

    if scale_method and scale_method not in ['normal','cap']:
        raise ValueError("extreme_method must be one of ['normal','cap'] or False")

    p_extreme_process = partial(extreme_process, num=extreme_num, method=extreme_method)
    p_scale_process = partial(scale_process, method=scale_method)
    all_funcs = [p_extreme_process, p_scale_process]

    if not funcs:
        funcs = []
    all_funcs.extend(funcs)

    if not extreme_method:
        all_funcs.remove(p_extreme_process)

    if not scale_method:
        all_funcs.remove(p_scale_process)

    if all_funcs:
        for func in all_funcs:
            raw_data = raw_data.groupby(level=0).apply(func)

    if not all_funcs:
        print("未定义任何数据处理函数，返回原始数据！")

    return raw_data
# File:      /opt/conda/lib/python3.5/site-packages/WindAlpha/analysis.py
# Type:      function